
# Run
- This notebook is entirelly sql; rec'd running against serverless sql warehouse for best experience.

In [1]:
%sql
-- Daily and Monthly Active Users
select
  distinct event_date as `Date`,
  date_format(event_date, 'yyyy-MM') AS year_month,
  workspace_id,
  user_identity.email as user_email
from
  system.access.audit

,Date,year_month,workspace_id,user_email
0,2024-11-08,2024-11,0,siddhesh.pore@databricks.com
1,2024-11-08,2024-11,0,d0423bc4-ecd2-4ac8-bd93-c40d989956f6
2,2024-11-08,2024-11,0,ritesh.patel@databricks.com
3,2024-11-08,2024-11,2556758628403379,eric.ferreira@databricks.com
4,2024-11-08,2024-11,0,roy.bang@databricks.com
5,2024-11-08,2024-11,0,suman.misra@databricks.com
6,2024-11-08,2024-11,0,eric.peter@databricks.com
7,2024-11-08,2024-11,0,3ca21e2b-45b3-442e-8278-0ea811229cf1
8,2024-11-08,2024-11,0,nishant.deshpande@databricks.com
9,2024-11-08,2024-11,0,yatish.anand@databricks.com


In [2]:
%sql
-- Sum DBUs by user last 30 days
SELECT
  sku_name,
  -- usage_metadata.job_id,
  -- usage_metadata.notebook_id,
  SUM(usage_quantity) as total_dbu
FROM
  system.billing.usage
WHERE
  identity_metadata.run_as = 'juan.lamadrid@databricks.com'
  -- and billing_origin_product in ('JOBS','INTERACTIVE')
  -- and product_features.is_serverless -- for serverless only
  and usage_unit = 'DBU'
  and usage_date >= DATEADD(day, -30, current_date)
GROUP BY all
ORDER BY
  total_dbu DESC


,sku_name,total_dbu
0,ENTERPRISE_JOBS_SERVERLESS_COMPUTE_US_WEST_OREGON,243.507579245804739818
1,ENTERPRISE_SERVERLESS_REAL_TIME_INFERENCE_US_WEST_OREGON,45.500000000000000000
2,ENTERPRISE_JOBS_COMPUTE_(PHOTON),18.165562944444444400
3,ENTERPRISE_ALL_PURPOSE_SERVERLESS_COMPUTE_US_WEST_OREGON,1.411592025000000020


In [0]:
-- Report on DBUs consumed by a particular user
-- https://docs.databricks.com/en/admin/system-tables/serverless-billing.html#report-on-dbus-consumed-by-a-particular-user

SELECT
  usage_metadata.job_id,
  usage_metadata.notebook_id,
  SUM(usage_quantity) as total_dbu
FROM
  system.billing.usage
WHERE
  identity_metadata.run_as = 'juan.lamadrid@databricks.com'
  and billing_origin_product in ('JOBS','INTERACTIVE')
  and product_features.is_serverless -- SERVERLESS
  and usage_unit = 'DBU'
  and usage_date >= DATEADD(day, -30, current_date)
GROUP BY
  1,2
ORDER BY
  total_dbu DESC